# Download objects and pathes form various sources

The data collected uses data from Austria. The used sources can be used for other countries as well. 

**Generally the data is not guaranteed to be complete or accurate.**


## Wikipedia

Data is downloaded to find addresses and match coordinates. This helps in geolocating various points of interest.

Trainstations parsed from wikipedia are not used, the data quatility of OSM is better

## OpenStreetMap

Data is downloaded via overpass queries The results are points of interests (and pathes)

See https://overpass-turbo.eu/# for checking and constructing queries

<!-- ## LLM Services -->

<!-- Are used to check some of the data, and impute missing values (e.g. coordinates) -->

## Postprocessing Steps

The raw data is generally cached when possible, to avoid unnecessary requests and improve performance.


- **Filtering**: Removing irrelevant or duplicate data entries.

- **Normalizing**: Standardizing data formats for consistency.

- **Storing**: Saving the processed data in JSON format for easy access and manipulation.


## Additional Resources:

* GIS Austria -> https://gis.stmk.gv.at/wgportal/atlasmobile/map/Basiskarten/Basiskarte

    Also flooding zones are available here

* https://www.geonames.org/

* statistik.at



In [ ]:
import numpy as np
import pandas as pd
import geopy.distance as gd

import requests 
from bs4 import BeautifulSoup 
import json
import os
import tqdm

from datetime import datetime
import time
import datautils

import simplekml
import pandas as pd
import zipfile

from fastkml import kml


# create folder structures and holds constants
data_path = datautils.DataPaths("../data")

In [ ]:
# get the response in the form of html
wikiurl = "https://de.wikipedia.org/wiki/Liste_der_Eisenbahnh%C3%B6fe_und_-haltestellen_in_%C3%96sterreich"
table_class = "wikitable sortable jquery-tablesorter"
response = requests.get(wikiurl)
print(response.status_code)
soup = BeautifulSoup(response.text, "html.parser")
wikitables = soup.find_all("table", {"class": "wikitable"})

print(f" # of tables {len(wikitables)}")
# print(wikitables[1])

# trainstation_table
pd_trainstations_partitial = []
for index, wikitable in enumerate(wikitables):
    if index == 0:
        print("skip first")
        continue
    df = pd.read_html(str(wikitable), header=0)[0]
    pd_trainstations_partitial.append(df)

print(f"parsed tables {len(pd_trainstations_partitial)}")
pd_trainstations = pd.concat(pd_trainstations_partitial, ignore_index=True)

print(f"columns {pd_trainstations.columns}")

In [ ]:
pd_trainstations

In [23]:
len(pd_trainstations)
current_year = datetime.now().year
pd_trainstations.to_csv(f"{data_path.wikipedia}/austria_trainstations_{datetime.now().year}.csv", sep=';')
pd_trainstations_styria = pd_trainstations.query("BL == 'St'")


In [24]:
#http://nominatim.openstreetmap.org/search?q=Wien,+Austria&format=json&polygon=1&addressdetails=1
server = 'https://nominatim.openstreetmap.org'
#s = requests.session()
#s.config['keep_alive'] = False
#headers = {'Accept': 'application/json', 'Content-Type': 'application/json', "user-agent": "python-requests/2.9.1", "HTTP Referer": "http://nominatim.openstreetmap.org"}
headers = {"user-agent": "python-requests/2.9.1"}

def find_coords(name:str):
    print(name)
    esc_name = name.replace(' ', '+')
    print(esc_name)
    url = f"{server}/search?q={esc_name},Österreich&format=json&polygon=1&addressdetails=1"
    print(url)
    response = requests.get(url, headers=headers)

    #response = requests.get("%s/search?q=%s,Österreich&format=json&polygon=1&addressdetails=1" % (server, esc_name), headers=headers)

    if not response.status_code == 200:
        print("error: service not reachable; error:", response.status_code)
        exit(1)

    #headers_json = {'Accept': 'application/json', 'Content-Type': 'application/json', 'token': token}
    #with open(artifacts_file, 'rb') as f:
    #    data = f.read()

    #response = requests.post("%s/secure/asset/clientupdate" % server, headers=headers_json, data=data)

    #json_request_content = json.loads(data)
    #json_request_content.pop("installedModules", None)

    print(response.text)

    json_content = json.loads(response.text)
    if len(json_content) == 0:
        return (False, 0, 0)
    
    lat = (float)(json_content[0]['lat'])
    lon = (float)(json_content[0]['lon'])
    bb = (json_content[0]['boundingbox'])
    height = float(bb[1])-float(bb[0])
    width = float(bb[3])-float(bb[2])

    print(len(json_content))
    
    return (True, lat, lon, height, width)

In [25]:
skip_trainstations = True
not_found_trainstations = []
found_coords = {}
found_coords_list = []

# reset X and Y
pd_trainstations['X'] = float('Nan')
pd_trainstations['Y'] = float('Nan')

if skip_trainstations == False:
    # iterate all trainsations in Styria and find the coordinates
    for index, trainstation in pd_trainstations[pd_trainstations["BL"] == "St"].iterrows():
        station_name = trainstation["Name"]

        while True:
            search_station = (station_name + ",") if len(station_name) > 0 else ""
            print(index, search_station)
            search = f"Bahnhof,{search_station}{trainstation['Standort­gemeinde']}"

            coords = find_coords(f"{search}")

            if coords[0]:
                found_coords[trainstation["Name"]] = [coords[1], coords[2]]
                found_coords_list.append([coords[1], coords[2]])
                print(coords)
                pd_trainstations.at[index, "X"] = coords[1]
                pd_trainstations.at[index, "Y"] = coords[2]
                break
            else:
                had_split_char = False
                if ", " in station_name:
                    station_name = station_name.split(", ", 1)[1]
                    had_split_char = True
                elif " " in station_name:
                    station_name = station_name.split(" ", 1)[1]
                    had_split_char = True

            if len(station_name) == 0:
                print("not found")
                not_found_trainstations.append((trainstation["Name"], trainstation["Art"]))
                found_coords_list.append([float("Nan"), float("Nan")])
                break

            if had_split_char == False:
                station_name = ""
                continue

        time.sleep(0.8) # needed by the provider of the server

In [ ]:
print(not_found_trainstations)

In [ ]:
pd_trainstations.query("BL == 'St'")

In [ ]:
pd_trainstations.sort_values(by='X')

In [29]:
def download_poi(query: str, tagname: str, use_cache: bool = True):
    server = "https://lz4.overpass-api.de/api/interpreter"
    query = "\n".join(
        [line for line in query.split("\n") if not line.strip().startswith("//")]
    )
    esc_query = query.replace("\n", "").replace(" ", "+")

    # not perfekt to set the object here, but better than link it on every call
    cache_dir = f"{data_path.cache}/boundary/place"
    datautils.create_folders([cache_dir])

    if os.path.exists(cache_dir) == False:
        os.makedirs(cache_dir)

    cache_file = os.path.join(cache_dir, f"{tagname}.json")

    if use_cache and os.path.exists(cache_file):
        # if os.path.exists(cache_file):
        with open(cache_file, "r", encoding="utf8") as f:
            data = f.read()
            print(f"returning data '{cache_file}' from cache ({len(data)} bytes)")
            return data

    url = f"{server}?data={esc_query}"
    print(url)
    headers = {"user-agent": "python-requests/2.9.1"}
    response = requests.get(url, headers=headers)

    if not response.status_code == 200:
        print(
            f"error: service not reachable; error: {response.status_code}, {response}"
        )
        return None

    # store
    content = response.text
    json_content = json.loads(content)

    with open(cache_file, "w", encoding="utf8") as f:
        f.write(content)
        print(f"saved to cache file '{cache_file}' from cache ({len(content)} bytes)")

    return content

In [30]:
def collect_way_coords(content, result):
    #id = content["id"]
    type_ = content["type"]


    if type_ == "way":
        for g in content["geometry"]:
            result.append([g["lat"], g["lon"]])
    elif type_ == "relation":
        for m in content["members"]:
            collect_way_coords(m, result)


def json_way_to_pd(content: str,):
    content_json = json.loads(content)

    result = []

    element_count = len(content_json["elements"])
    if element_count == 0:
        print(f"element has no entries")
        return None

    for e in content_json["elements"]:
        collect_way_coords(e, result)

    df = pd.DataFrame(result)
    df.columns = ["lat", "lon"]
    return df

def json_nodes_to_pd(content: str, skip_nodes_without_tags: bool = True):
    content_json = json.loads(content)

    result = []

    element_count = len(content_json["elements"])
    if element_count == 0:
        print(f"element has no entries")
        return None

    for e in content_json["elements"]:
        id = e["id"]
        type_ = e["type"]
        if type_ == "node":
            lat = e["lat"]
            lon = e["lon"]
        elif type_ == "way":
            lat = e["center"]["lat"]
            lon = e["center"]["lon"]
        else:
            print(f"skipping node {id} {type_}")
            continue

        tags = e.get("tags", None)
        if tags is None:
            name = None
            alt_name = None

            # skip entries without tags, most likely they are 'node's of a 'way', and therefore are duplicates
            if skip_nodes_without_tags:
                continue
        else:
            name = tags.get("name", None)
            alt_name = tags.get("alt_name", None)

        result.append([id, lat, lon, name, alt_name])

    df = pd.DataFrame(result)
    df.columns = ["id", "lat", "lon", "name", "alt_name"]
    return df

In [31]:
def building_centers(content: str):
    content_json = json.loads(content)

    result = []
    nodes = {}
    element_count = len(content_json["elements"])
    if element_count == 0:
        # print("# %s has no element entries" % (name ))
        return None

    for e in content_json["elements"]:
        # print(e)
        id = e["id"]
        type_ = e["type"]
        if type_ == "node":
            lat = e["lat"]
            lon = e["lon"]
            nodes[id] = (lat, lon)
        elif type_ == "way":
            building = e["tags"]["building"]
            address = ""
            node_coordinates = []
            lat = 0
            lon = 0
            node_count = len(e["nodes"])
            for way_node_id in e["nodes"]:
                way_node = nodes[way_node_id]
                lat += way_node[0] / node_count
                lon += way_node[1] / node_count
                node_coordinates.append([way_node[0], way_node[1]])

            result.append([id, lat, lon, building, address])

        else:
            print("skipping node %s %s" % (id, type_))
            continue

    df = pd.DataFrame(result)
    df.columns = ["id", "lat", "lon", "building", "address"]

    return df

In [ ]:
query = """
[out:json];
area[name=\"Steiermark\"]->.b;
(
nwr(area.b)[highway=bus_stop];
nwr(area.b)[public_transport=stop_position];
nwr(area.b)[public_transport=platform];
);
(._;>;);
out center;
"""

tag = "bus_stops_styria"

data = json_nodes_to_pd(download_poi(query, tag, use_cache=True))
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
query = """
[out:json];
area[name="Steiermark"]->.b;
(
//node(area.b)[railway="^(halt|station|stop)$"];
nwr(area.b)[railway="halt"];
nwr(area.b)[railway="station"];
nwr(area.b)[railway="stop"];
);
out center;
"""

tag = "railway_stations_styria"

data = json_nodes_to_pd(download_poi(query,tag, use_cache=True))
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
query = """
[out:json];
area[name=\"Steiermark\"]->.b;
(
nwr(area.b)[shop=supermarket];
//way(area.b)[shop=supermarket];
);
(._;>;);
out center;
"""

tag = "supermarkets_styria"
data = json_nodes_to_pd(download_poi(query,tag, use_cache=True))
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
query = """
[out:json];
area[name=\"Steiermark\"]->.b;
(
nwr(area.b)[amenity=pharmacy];
//way(area.b)[amenity=pharmacy];
);
(._;>;);
out center;
"""

tag = "pharmacy_styria"
data = json_nodes_to_pd(download_poi(query,tag, use_cache=True))
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:

query = """
[out:json];
area[name=\"Steiermark\"]->.b;
(
nwr(area.b)[shop=\"chemist\"]; 
);
(._;>;);
out center;
"""
# drogerie in german
tag = "chemist_styria"
data = json_nodes_to_pd(download_poi(query,tag, use_cache=True))
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
query = """
[out:json];
area[name=\"Steiermark\"]->.b;
(
nwr(area.b)[shop=bakery];
//way(area.b)[shop=bakery];
);
(._;>;);
out center;
"""


tag = "bakery_styria"
data = json_nodes_to_pd(download_poi(query,tag, use_cache=True))
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
#restaurants

query = """
[out:json];
area[name=\"Steiermark\"]->.b;
(
nwr(area.b)[amenity=restaurant];
//way(area.b)[amenity=restaurant];
);
(._;>;);
out center;
"""

tag = "restaurant_styria"
data = json_nodes_to_pd(download_poi(query,tag, use_cache=True))
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
#shops
# without supermarkets and bakeries
query = """
[out:json];
area[name="Steiermark"]->.b;
((
nwr(area.b)[shop]; 
 - nwr(area.b)[shop="supermarket"];
);
- nwr(area.b)[shop="bakery"];

);

out center;
"""

tag = "shops_styria"
data = json_nodes_to_pd(download_poi(query,tag, use_cache=True))
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
# Motorway exits
query = """
[out:json];
area[name=\"Steiermark\"]->.b;
// Select motorway links
way(area.b)[highway="motorway_link"]->.motorway_links;

// Select primary roads

way(area.b)[highway~"^(primary|secondary|tertiary|trunk|trunk_link|primary_link|secondary_link|tertiary_link|unclassified)$"]->.primary_roads;

// Find intersecting nodes
node(w.motorway_links)->.motorway_nodes;
node(w.primary_roads)->.primary_nodes;
node.motorway_nodes.primary_nodes;

// Output the intersecting nodes
out body;
"""

tag = "motorway_exits"
data = json_nodes_to_pd(download_poi(query,tag,use_cache=False),skip_nodes_without_tags=False)
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
# get gas stations
#nwr -> is type agnostic and selects nodes, ways and relations

query ="""
[out:json];
area[name="Steiermark"]->.b;
(
//nwr -> is type agnostic and selects nodes, ways and relations
nwr (area.b)[amenity=fuel];
//node(area.b)[amenity=fuel];
//way(area.b)[amenity=fuel];
);
(._;>;);
out center;
"""

tag = "gasstations_styria"
data = json_nodes_to_pd(download_poi(query,tag, use_cache=True))
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
# motorways (Autobahnen und Zuführer)
# query ="""
# [out:json];
# area[name="Steiermark"]->.b;
# way(area.b)[highway~"^(motorway|trunk|motorway_link|trunk_link)$"]->.hw;
# node(w.hw);
# //(._;>;);
# out body;
# """

query = """
[out:json];
area[name="Steiermark"]->.b;
way(area.b)[highway~"^(motorway|trunk|motorway_link|trunk_link)$"][!"tunnel"]->.hw;
node(w.hw);
out body;
"""

tag = "motorway_styria"
data = json_nodes_to_pd(download_poi(query,tag, use_cache=False),skip_nodes_without_tags=False)
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
# motorways (Autobahnen und Zuführer)
query ="""
[out:json];
area[name="Steiermark"]->.b;
way(area.b)[highway~"^(primary|secondary)$"][!"tunnel"]->.hw;
node(w.hw);
//(._;>;);
out body;
"""

highway="motorway"
tunnel="yes"

tag = "street_primary_secondary_styria"
data = json_nodes_to_pd(download_poi(query,tag,use_cache=False),skip_nodes_without_tags=False)
data.to_csv(f"{data_path.osm}/{tag}.csv")


In [ ]:
# soccer field/stadium

query = """
[out:json];
area[name="Steiermark"]->.b;
(
nw(area.b)[sport=soccer];
);
(._;>;);
out center;
"""

tag = "soccer_fields_styria"
data = json_nodes_to_pd(
    download_poi(query, tag, use_cache=True), skip_nodes_without_tags=False
)
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
import requests
from shapely.geometry import Polygon
from area import area as calculate_geo_area
import csv

def calculate_area(data):
    # Extract nodes and ways from the data
    nodes = {el['id']: el for el in data['elements'] if el['type'] == 'node'}
    ways = [el for el in data['elements'] if el['type'] == 'way']

    areas = []

    for way in ways:
        coords = [(nodes[node_id]['lat'], nodes[node_id]['lon']) for node_id in way['nodes']]
        # Create GeoJSON-like structure
        geojson_polygon = {
            'type': 'Polygon',
            'coordinates': [[(lon, lat) for lat, lon in coords]]
        }
        # Calculate area using the 'area' library
        area_size = calculate_geo_area(geojson_polygon)
        polygon = Polygon([(lon, lat) for lat, lon in coords])
        areas.append({
            'id': way['id'],
            'center_lat': polygon.centroid.y,
            'center_lon': polygon.centroid.x,
            'area': area_size
        })
        print(f"Area of way {way['id']}: {area_size} square meters")

    return areas

def save_to_csv(areas, filename='areas.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'center_lat', 'center_lon', 'area'])
        for area in areas:
            writer.writerow([area['id'], area['center_lat'], area['center_lon'], area['area']])

# Example usage
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area[name="Steiermark"]->.b;
(
  way(area.b)[sport=soccer];
);
(._;>;);
out body;
"""
response = requests.post(overpass_url, data=overpass_query)
data = response.json()

areas = calculate_area(data)
save_to_csv(areas, f"{data_path.osm}/soccer_fields_areas.csv")

In [ ]:
# railway rails

query = """
[out:json];
area[name="Steiermark"]->.b;
way(area.b)[railway=rail][!"tunnel"]->.rails;
node(w.rails);
//(._;>;);
out body;
"""

tag = "railway_rails_styria"
data = json_nodes_to_pd(
    download_poi(query, tag, use_cache=False), skip_nodes_without_tags=False
)
data.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
# rivers

query ="""
[out:json];
area[name="Steiermark"]->.b;
way(area.b)[waterway=river]->.rivers;
node(w.rivers);
//(._;>;);
out body;
"""

tag = "rivers_styria"
data = json_nodes_to_pd(download_poi(query,tag,use_cache=False),skip_nodes_without_tags=False)
data.to_csv(f"{data_path.osm}/{tag}.csv")
    

In [ ]:
# streams

query ="""
[out:json];
area[name="Steiermark"]->.b;
way(area.b)[waterway=stream]->.streams;
node(w.streams);
//(._;>;);
out body;
"""

tag = "streams_styria"
data = json_nodes_to_pd(download_poi(query,tag,use_cache=False),skip_nodes_without_tags=False)
data.to_csv(f"{data_path.osm}/{tag}.csv")


In [ ]:
# houses
#13.6054501606841 16.1110951871948
y1= 13.5
y2 = 16.2
#46.649784205005204 47.7718630862253
x1 = 46.6
x2 = 47.8

#[bbox:south,west,north,east]
#[bbox:50.6,7.0,50.8,7.3]
#"[bbox:%.1f,%.1f,%.1f,%.1f]" 

query ="""
[out:json];
nw[building](%.6f,%.6f,%.6f,%.6f);
(._;>;);
out center;
"""

x_step = (x2-x1)/20
y_step = (y2-y1)/20
print(x_step,y_step)
houses = []
for x in tqdm.tqdm(np.arange(x1, x2, x_step)):
    for y in np.arange(y1, y2, y_step):
        tag = "houses_styria_x=%.6f_y=%.6f,stepx=%.6f,stepy=%.6f" % (x,y,x_step,y_step)
        part_query = query % (x, y, x + x_step, y + y_step)
        #print(part_query)
        #data = json_nodes_to_pd(download_poi(part_query,tag))
        data = download_poi(part_query,tag, use_cache=True)
        houses.append(building_centers(data))
        
        #f data is not None:
        #   print(part_query)
#houses_all = np.concatenate(houses)

In [205]:
houses_pd = pd.concat(houses, ignore_index=True)
tag = "houses_styria_center"
houses_pd.to_csv(f"{data_path.osm}/{tag}.csv")

In [ ]:
x = houses_pd['building'].value_counts()
x
#len(x)

In [ ]:
houses_pd[~houses_pd['building'].isin(['shed', 'garage', 'garages', 'barn', 'cabin', 'hut', 'collapsed', 'cowshed', 'pavilion', 'shelter', 'greenhouse'])]


In [ ]:
# download all schools from  statistik austria
#https://data.statistik.gv.at/web/catalog.jsp
# https://www.data.gv.at/suche/?typeFilter%5B%5D=dataset&searchterm=schulen&searchin=data

# Schools in Austria
# https://www.data.gv.at/katalog/dataset/6f3e9528-33d5-37e4-ba0a-a42e041ad41d#resources

# Schools by type in Styria
# https://www.data.gv.at/katalog/dataset/7b86e565-cde3-4e18-804c-d175748e07b4

In [10]:
# download the kmz file, in case it isn't available anymore, a copy is in the data folder
url_school_locations = "https://service.stmk.gv.at/ogd/OGD_Data_ABT17/geoinformation/Bildungsstandorte.kmz"
response = requests.get(url_school_locations)
kmz_path = f"{data_path.statistik_austria}/Bildungsstandorte.kmz"
with open(kmz_path, 'wb') as f:
    f.write(response.content)

In [ ]:
import zipfile

from fastkml import kml

def extract_data_from_kml_record(record):
    einrichtung = None
    typ_lang = None
    for element in record.extended_data.elements:
        #print(element)
        for data in element.data:
            #print("dataline", data)
            if data.get("name","") == "EINRICHTUNG":
                einrichtung = data.get("value","")
            elif data.get("name","") == "TYP_LANG":
                typ_lang = data.get("value","")
    return einrichtung, typ_lang


# read KML content
kmz_file = f"{data_path.statistik_austria}/Bildungsstandorte.kmz"

kmz = zipfile.ZipFile(kmz_file, 'r')  # extract zip file first, then read kmz file inside the extracted folder

kml_content = kmz.open('doc.kml', 'r').read()  # kml content
# FIX CONTENT
kml_content_fixed = kml_content.replace(b'xsd:string', b'string')
kml_content_fixed = kml_content_fixed.replace(b'xsd:int', b'int')

print("len", len(kml_content_fixed))
# create KML object
k = kml.KML()
k.from_string(kml_content_fixed)

# read features from docs to folders to records and then extract geometries - in my case, Shapely points
docs = list(k.features())
folders = []
for d in docs:
    folders.extend(list(d.features()))
records = []
for f in folders:
    records.extend(list(f.features()))
geoms = [element.geometry for element in records]  # extract geometry
# Extract data from records
data = []
for record in records:
    name = record.name
    description = record.description
    coordinates = record.geometry.coords[0]
    einrichtung, typ_lang = extract_data_from_kml_record(record)

    data.append([name, einrichtung, typ_lang, coordinates[1], coordinates[0]])  # lat, lon

# Create a DataFrame
df_records = pd.DataFrame(data, columns=['Name', 'Type', 'Type_Long', 'Latitude', 'Longitude'])
print(df_records)
# # plot the geoms - latitude (y), longitude (x)
# import folium
# import pandas as pd
# mapit = folium.Map()
# for geom in geoms:
#     #print(geom)
#     folium.Marker(location=(geom.y, geom.x), fill_color='#43d9de', radius=2).add_to(mapit)
# mapit.save('map.html')

In [8]:
df_records[(df_records['Type'] == 'Schule')].to_csv(f"{data_path.statistik_austria}/schools_styria.csv")
df_records[(df_records['Type_Long'] == 'Volksschule')].to_csv(f"{data_path.statistik_austria}/elementary_schools_styria.csv")

secondary_types_german = ["Mittelschule", "Allgemeinbildende höhere Schule","Lehranstalt für wirtschaftliche Berufe, Sozialberufe und Tourismus", "Polytechnische Schule", "Land- und forstwirtschaftliche mittlere Schule", "Kaufmännische mittlere oder höhere Schule", "Technisch-gewerbliche Schule"]
#secondary (or middleschool)
df_records[(df_records['Type_Long'].isin(secondary_types_german))].to_csv(f"{data_path.statistik_austria}/secondary_schools_styria.csv")


In [ ]:
# administrative border of styria for masking the region

query ="""
[out:json];
area[name="Steiermark"]->.b;
(
	wr(area.b)[boundary="administrative"][admin_level=4];
);
out geom;
"""

tag = "admin_border_styria"
data = json_way_to_pd(download_poi(query,tag))
data.to_csv(f"{data_path.osm}/{tag}.csv")

